In [1]:
import os

if not os.path.isfile("./input.txt"):
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/refs/heads/master/data/tinyshakespeare/input.txt

--2025-06-26 16:40:50--  https://raw.githubusercontent.com/karpathy/char-rnn/refs/heads/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.006s  

2025-06-26 16:40:50 (185 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm

# hyperparams
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd = 384
n_layer = 6
n_head = 6
dropout = 0.2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device', device)

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

def encode(s):
    return [stoi[c] for c in s]

def decode(t_ids):
    return "".join([itos[i] for i in t_ids])

# train val splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # since tril is not a param, in pytorch convension it should be made a buffer
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x) # (B,T,head_size)
        q = self.query(x) # (B,T,head_size)
        # compute attension scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B,T,head_size) @ (B,head_size,T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B, T, C)
        out = wei @ v #(B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([ Head(head_size) for _ in range(num_heads) ])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

class FeedForward(nn.Module):
    """ a simple linear layer followed by non linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # projection layer going back into residual pathway
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # notice `x +` residual connections
        return x + self.ffwd(self.ln2(x))

# simple bigram model
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embdedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(* [ Block(n_embd, n_head=n_head) for _ in range(n_layer) ])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size) # Language modeling head

    def forward(self, idx, targets=None):
        # idx and targets both (B,T)
        B, T = idx.shape
        tok_emb = self.token_embdedding_table(idx)  # (B,T,n_embed) - C is channels in this case len of vocab_size
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B,T,C) pos_emb will is broadcasted to (1,T,C)
        x = self.blocks(x) # apply one head of self-attension (B, T, C)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is not None:
            # torch's cross entropy expects input in (B, C, T) format so
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop the idx into last block_size tokens
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            # pluck out only on the last time step
            logits = logits[:, -1, :]  # (B, C)
            probs = F.softmax(logits, dim=-1)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to running sequence
            idx = torch.cat([idx, idx_next], dim=1)  # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)

print(sum(p.numel() for p in m.parameters()) / 1e6, "M parameters")

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in tqdm(range(1, max_iters+1), ncols=100):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"\nstep {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch("train")

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

open("./output.txt", "w").write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

torch.save({
        "model_state_dict": model.state_dict(),
        "n_layer": n_layer,
        "n_head": n_head,
        "block_size": block_size,
    }, "./nano-gpt-char.pth")

device cuda
10.788929 M parameters


 10%|█████▋                                                   | 500/5000 [04:57<25:42:20, 20.56s/it]


step 500: train loss 2.0078, val loss 2.0908


 20%|███████████▏                                            | 1000/5000 [09:58<22:44:25, 20.47s/it]


step 1000: train loss 1.6018, val loss 1.7673


...

 90%|███████████████████████████████████████████████████▎     | 4500/5000 [45:06<2:50:21, 20.44s/it]


step 4500: train loss 1.1106, val loss 1.4771


100%|███████████████████████████████████████████████████████████| 5000/5000 [50:07<00:00,  1.66it/s]


step 5000: train loss 1.0761, val loss 1.4852



Better than it were by thee.

ISABELLA:
I won; I must so; were o' that were as 'I more said
Where my own mind 's lew night,
It must I might perfect most you be his design.

ANTIGONUS:
He's slaid: I heard of them mock my law:
To London the bleed your own when he flesh is
's now the teeth my son werefore.

FROTHER:
So dost this is tarding? is not Acroriof?

WARWICK:
Yes, gentlemen, then speak; they are chein they have:
They like my uncle, instant, I can ceremons
You high to bear the muldest gostne
